In [1]:
import os
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

In [5]:
questions_dir = 'D:\Projects\Companies Test\Pyxiscience\enonces\enonces'
solutions_dir = 'D:\Projects\Companies Test\Pyxiscience\solutions\solutions'

# Initialisation du modèle GPT
llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Exemple de prompt pour QCM
qcm_prompt = PromptTemplate(
    input_variables=["question", "solution"],
    template="""
    **Transform the following question and solution into a multiple-choice question:**
    Question: In a town, there are four bakeries. Each and everyone of them has to close one day a week. Determine the number of ways there are to assign a weakly day of closing.
    solution: For each bakery, there are 7 possible choices. Thus, one has $$7^4=2401$$ different ways to assign a weekly closing day to each bakery.
    
    Question: In a town, there are four bakeries. Each one of them has to close one day a week. How many different ways are there to assign a weekly closing day to each bakery?
    Answer Choices:
    a) 7×4=28
    b) 7^4=2401
    c) 4^7=16384
    d) 4!=24
    Solution: The correct answer is b) 7^4=2401. For each bakery, there are 7 possible choices for the closing day, leading to a total of 7^4=2401 different ways to assign a weekly closing day to each bakery.

    **Transform the following question and solution into a multiple-choice question:**

    Question: {question}
    Solution: {solution}
    """
)

# Exemple de prompt pour Exercice à Trou
fill_in_the_blank_prompt = PromptTemplate(
    input_variables=["question", "solution"],
    template="""
    **Transform the following question and solution into a fill-in-the-blank exercise:**
    Question: In a town, there are four bakeries. Each and everyone of them has to close one day a week. Determine the number of ways there are to assign a weakly day of closing.
    solution: For each bakery, there are 7 possible choices. Thus, one has $$7^4=2401$$ different ways to assign a weekly closing day to each bakery.

    Question: In a town, there are four bakeries. Each one of them has to close one day a week. Complete the following statements to determine the number of ways to assign a weekly closing day to each bakery.
    - For each bakery, there are ________ possible closing days.
    - Therefore, for four bakeries, the total number of possible combinations is ________.
    - The total number of ways to assign a weekly closing day to each bakery is ________.
    Solution:
    - For each bakery, there are 7 possible closing days.
    - Therefore, for four bakeries, the total number of possible combinations is 7^4.
    - The total number of ways to assign a weekly closing day to each bakery is 2401.

    **Transform the following question and solution into a fill-in-the-blank exercise:**
    Question: {question}
    Solution: {solution}
    """
)

# Initialisation des chaînes
qcm_chain = LLMChain(llm=llm, prompt=qcm_prompt)
fill_in_the_blank_chain = LLMChain(llm=llm, prompt=fill_in_the_blank_prompt)


In [3]:
class FileReadingAgent:
    def __init__(self, questions_dir, solutions_dir):
        self.questions_dir = questions_dir
        self.solutions_dir = solutions_dir
    
    def read_markdown_files(self, directory):
        texts = {}
        for filename in os.listdir(directory):
            if filename.endswith('.md'):
                with open(os.path.join(directory, filename), 'r') as file:
                    md_text = file.read()
                    texts[filename] = md_text
        return texts

    def get_questions_and_solutions(self):
        questions_texts = self.read_markdown_files(self.questions_dir)
        solutions_texts = self.read_markdown_files(self.solutions_dir)
        return questions_texts, solutions_texts

class QCMGenerationAgent:
    def __init__(self, qcm_chain):
        self.qcm_chain = qcm_chain
    
    def generate_qcm(self, question_text, solution_text):
        return self.qcm_chain.run(question=question_text, solution=solution_text)
    
class FillInTheBlankGenerationAgent:
    def __init__(self, fill_in_the_blank_chain):
        self.fill_in_the_blank_chain = fill_in_the_blank_chain
    
    def generate_fill_in_the_blank(self, question_text, solution_text):
        return self.fill_in_the_blank_chain.run(question=question_text, solution=solution_text)
    
class AgentManager:
    def __init__(self, file_reader, qcm_agent, fill_in_the_blank_agent):
        self.file_reader = file_reader
        self.qcm_agent = qcm_agent
        self.fill_in_the_blank_agent = fill_in_the_blank_agent
    
    def process(self):
        questions_texts, solutions_texts = self.file_reader.get_questions_and_solutions()
        
        for question_file, question_text in questions_texts.items():
            # Trouver le fichier de solution correspondant
            solution_file = question_file.replace('question', 'solution')
            if solution_file in solutions_texts:
                solution_text = solutions_texts[solution_file]
                
                # Générer QCM et exercice à trous
                qcm_result = self.qcm_agent.generate_qcm(question_text, solution_text)
                fill_in_the_blank_result = self.fill_in_the_blank_agent.generate_fill_in_the_blank(question_text, solution_text)
                
                # Sauvegarde des résultats
                self.save_results(question_file, qcm_result, fill_in_the_blank_result)
    
    def save_results(self, question_file, qcm_result, fill_in_the_blank_result):
        os.makedirs('qcm', exist_ok=True)
        os.makedirs('fill', exist_ok=True)
        
        with open(f'qcm/qcm_{question_file}', 'w') as f:
            f.write(qcm_result)
        
        with open(f'fill/fill_in_the_blank_{question_file}', 'w') as f:
            f.write(fill_in_the_blank_result)




In [6]:
# Initialiser les agents
file_reader = FileReadingAgent(questions_dir, solutions_dir)
qcm_agent = QCMGenerationAgent(qcm_chain)
fill_in_the_blank_agent = FillInTheBlankGenerationAgent(fill_in_the_blank_chain)

# Créer le gestionnaire d'agents
agent_manager = AgentManager(file_reader, qcm_agent, fill_in_the_blank_agent)

# Lancer le traitement
agent_manager.process()
